In [1]:
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Flatten, LSTM, Dropout
from keras.utils import np_utils
from sklearn.preprocessing import MinMaxScaler
import math
import random
import time
import warnings
warnings.filterwarnings('ignore')

/Users/ddzina/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
test_data = pd.read_csv('test.csv')
test_sequences = test_data['Sequence']
chars = [',', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
y_pred = [0] * len(test_sequences)
exceptions = 0
n_train = 100

total_start = time.time()
for (j, sequence) in enumerate(test_sequences[:20]):
    start = time.time()
    if j % 1 == 0:
        print('Processing sequence {0}... '.format(j))
    
    try:
        last_comma = sequence.rindex(',')
        expected = sequence[last_comma:]
        sequence = sequence[:last_comma]
    except:
        continue
    
    if len(sequence) < 101:
        continue
#     numbers = sequence.split(',')
#     if len(numbers) < 4:
#         continue
#     len_n_train = 10
#     if len(numbers) < 11:
#         len_n_train = len(numbers) // 2
#     len_sequence = [len(item) for item in numbers]
#     len_x_train = []
#     len_y_train = []
#     for i in range(len(numbers) - len_n_train - 1):
#         len_x_train.append(numbers[i:i + len_n_train])
#         len_y_train.append(numbers[i + len_n_train])
#     print('formed len train data')
#     len_x_train = np.reshape(len_x_train, (-1, len_n_train, 1))
#     len_y_train = np_utils.to_categorical(len_y_train)
#     len_model = Sequential()
#     len_model.add(LSTM(4, input_shape=(len_x_train.shape[1], len_x_train.shape[2])))
#     len_model.add(Dropout(0.2))
#     len_model.add(Dense(len_y_train.shape[1], activation='softmax'))
#     len_model.compile(loss='categorical_crossentropy', optimizer='adam')
#     len_model.fit(len_x_train, len_y_train, epochs=20, verbose=1)    
#     print('len model fitted')
#     len_prob = len_model.predict(np.reshape(numbers[-len_n_train:], (1, len_n_train, 1)), verbose=1)
#     predicted_len = np.argmax(len_prob)
#     print('predicted_len:', predicted_len)
    predicted_len = 2

    sequence += ','
    x_train = []
    y_train = []
    for i in range(len(sequence) - n_train - 1):
        x_train.append([char_to_int[char] for char in sequence[i:i + n_train]])
        y_train.append(char_to_int[sequence[i + n_train]])

    print('formed train data')
    x_train = np.reshape(x_train, (-1, n_train, 1))
    y_train = np_utils.to_categorical(y_train)
    model = Sequential()
    model.add(LSTM(256, input_shape=(x_train.shape[1], x_train.shape[2])))
    model.add(Dropout(0.2))
    model.add(Dense(y_train.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    model.fit(x_train, y_train, epochs=20, verbose=0)
    print('model fitted')

    word = ''
    seq_to_predict = [char_to_int[char] for char in sequence[-n_train:]]
    for i in range(predicted_len):
        try:
            x_test = np.reshape(seq_to_predict, (1, n_train, 1))
            prediction = model.predict(x_test, verbose=0)
            pred_int = np.argmax(prediction)
            pred_char = int_to_char[pred_int]
            if pred_char == ',':
                break
            word += pred_char
            seq_to_predict.append(pred_int)
            seq_to_predict = seq_to_predict[-n_train:]
        except Exception as e:
            print(e)
    y_pred[j] = word
    end = time.time()
    if j % 1 == 0:
        print('Predicted: {0}, expected: {2}. Time: {1}'.format(word, end - start, expected))

total_end = time.time()
print('Total time: {0}'.format(total_end - total_start))

Processing sequence 0... 
Processing sequence 1... 
formed train data
model fitted
Predicted: 11, expected: ,257733967693. Time: 41.59799098968506
Processing sequence 2... 
formed train data
model fitted
Predicted: 11, expected: ,158. Time: 29.729705810546875
Processing sequence 3... 
formed train data
model fitted
Predicted: 33, expected: ,406. Time: 40.789715051651
Processing sequence 4... 
formed train data


KeyboardInterrupt: 

In [61]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [3]:
train_sequences = pd.read_csv('train.csv')['Sequence'].values
test_sequences = pd.read_csv('test.csv')['Sequence'].values

In [36]:
chars = ['-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ',']
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
[char_to_int[item] for item in numbers[i:i + len_n_train]]

In [39]:
def get_array(number):
    arr = [0] * 11
    arr[number] = 1
    return arr

In [56]:
len_n_train = 10
len_x_train = []
len_y_train = []
len_x_test = []
len_y_test = []
for sequence in np.hstack((train_sequences, test_sequences)):
#     if random.random() < 0.5:
#         continue
    numbers_str = sequence.split(',')
    if len(numbers_str) < 11:
        continue
    numbers = numbers_str
#     for number in numbers_str:
#         numbers.append([get_array(char_to_int[item]) for item in number])
    for i in range(len(numbers) - len_n_train - 2):
        len_x_train.append([len(number) for number in (numbers[i:i + len_n_train])])
        len_y_train.append(len(numbers[i + len_n_train]))
    len_x_test.append([len(number) for number in numbers[-len_n_train - 1:-1]])
    len_y_test.append(len(numbers[len(numbers) - 1]))

In [44]:
len_x_train[0]

[1, 1, 2, 2, 4, 5, 7, 9, 12, 16]

In [57]:
len_x_train = np.array(len_x_train)
len_x_test = np.array(len_x_test)
len_y_train = np.array(len_y_train)
len_y_test = np.array(len_y_test)

In [31]:
print(len_x_train[0])

[list([3]) list([5]) list([3, 5]) list([10, 9]) list([3, 2, 7, 5])
 list([4, 10, 7, 9, 8]) list([4, 3, 6, 3, 9, 5, 5])
 list([7, 2, 10, 3, 6, 9, 3, 2, 10])
 list([6, 2, 4, 3, 5, 7, 4, 9, 7, 5, 8, 7])
 list([3, 2, 9, 5, 5, 9, 8, 2, 7, 9, 6, 11, 2, 5, 9, 5])]


In [66]:
regressor = GradientBoostingRegressor(n_estimators=25)
regressor.fit(len_x_train, len_y_train)
len_y_pred = np.array([round(length) for length in regressor.predict(len_x_test)])

In [62]:
print('linear', sum(len_y_pred == len_y_test) / len(len_y_pred))

linear 0.7238008494389028


In [65]:
print('random forest', sum(len_y_pred == len_y_test) / len(len_y_pred))

random forest 0.8008299563591023


In [67]:
print('gradient boosting', sum(len_y_pred == len_y_test) / len(len_y_pred))

gradient boosting 0.6187412328553616


In [19]:
print(len_x_test[0])

['10' '18' '34' '34' '6' '26' '25' '24' '14' '13']


TypeError: Cannot cast array data from dtype('float64') to dtype('<U32') according to the rule 'safe'

In [114]:
n_train = 100
x_train = []
y_train = []
used_sequences = set()

for (j, sequence) in enumerate(np.hstack((train_sequences, test_sequences))):
    if len(sequence) < 101:
        continue
    if random.random() < 0.75:
        continue
    used_sequences.add(sequence)
    sequence += ','
    for i in range(len(sequence) - n_train - 1):
        x_train.append([char_to_int[char] for char in sequence[i:i + n_train]])
        y_train.append(char_to_int[sequence[i + n_train]])
x_train = np.reshape(x_train, (-1, n_train, 1))
y_train = np_utils.to_categorical(y_train)
print('formed train data')

formed train data


In [115]:
model = Sequential()
model.add(LSTM(256, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(x_train, y_train, epochs=2, verbose=1)
print('model fitted')

Epoch 1/2
4605355/4605355 [==============================] - 15030s 3ms/step - loss: 1.5617
Epoch 2/2
4605355/4605355 [==============================] - 13618s 3ms/step - loss: 1.4754
model fitted


In [116]:
total = 0
correct = 0
for i, seq in enumerate(train_sequences):
    if seq in used_sequences:
        continue
    try:
        last_comma = seq.rindex(',') + 1
    except:
        continue
    full_len = len(seq)
    expected = [char_to_int[char] for char in seq[last_comma:]]
    seq = seq[:last_comma]
    if len(seq) < 101:
        continue
    total += 1
    seq = [char_to_int[char] for char in seq]
    predicted = []
    while len(seq) < full_len:
        predicted_int = np.argmax(model.predict(np.array(seq[-n_train:]).reshape(-1, n_train, 1)))
        seq.append(predicted_int)
        predicted.append(predicted_int)
        if expected[:len(predicted)] != predicted:
            break
    if expected == predicted:
        correct += 1
    if i % 1000 == 0:
        print('total: {0}, correct: {1}.'.format(total, correct))

print('total: {0}, correct: {1}.'.format(total, correct))

total: 1, correct: 0.
total: 1917, correct: 192.
total: 3207, correct: 333.
total: 4506, correct: 476.
total: 5147, correct: 548.
total: 6430, correct: 680.
total: 7078, correct: 742.
total: 7713, correct: 806.
total: 10208, correct: 1089.
total: 11471, correct: 1201.
total: 13352, correct: 1416.
total: 14609, correct: 1548.
total: 15254, correct: 1628.
total: 15865, correct: 1689.
total: 16495, correct: 1751.
total: 17125, correct: 1828.
total: 18334, correct: 1965.
total: 18957, correct: 2017.
total: 19611, correct: 2085.
total: 20867, correct: 2247.
total: 22129, correct: 2395.
total: 22788, correct: 2470.
total: 24029, correct: 2602.
total: 24664, correct: 2655.
total: 25268, correct: 2721.
total: 27187, correct: 2932.
total: 27816, correct: 3004.
total: 28453, correct: 3070.
total: 29736, correct: 3223.
total: 30377, correct: 3303.
total: 31002, correct: 3388.
total: 32293, correct: 3530.
total: 32931, correct: 3608.
total: 33580, correct: 3683.
total: 36081, correct: 3943.
total: